In [1]:
# Feature extraction using gray level histogram

import numpy as np
import cv2 
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import h5py
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC 
from sklearn.metrics import accuracy_score

# label_dict = {'good':0, 'holes_cuts':1, 'threaderror':2, 'oilstains_colorerror':3, 'wrinkles':4, 'foreignbodies':5}
file_path = r"C:\Users\LENOVO\CV\Automated-Fabric-Defect-Inspection\textile_defect_data.hdF5"
imgs = h5py.File(file_path)['jute_defect_imgs'][:]
labels = h5py.File(file_path)['jute_defect_labels'][:]


def get_c6(hist):
    hist = hist.flatten()
    h_deriv = np.gradient(hist)
    h_deriv2 = np.gradient(h_deriv)
    numer = denom = 0
    for i in range(len(hist)):
        if h_deriv[i]>0:
            numer += h_deriv[i]*hist[i]
        if h_deriv2[i]<0:
            denom += h_deriv[i]*hist[i]
    return numer/denom

#box filter
filtered_images=[]
for img in imgs:
    kernel =np.ones((3,3),np.float32)/9
    filtered_image=cv2.filter2D(img,-1,kernel) #-1 argument specifies that the output image should have the same depth as the input image.
    filtered_images.append(filtered_image)
    
#histogram and feature vector
histogram=[]
feature_vectors=[]


    
for img in filtered_images:

    hist=cv2.calcHist([img],[0],None,[256],[0,255])
    hist_norm=hist/np.sum(hist)
#     print(hist_norm)
    c1=np.argmax(hist_norm)
    c2=np.min(hist_norm)
    c3=np.max(hist_norm)
    c4=(c3-c2)/255
    c5=c4/c1
    c6=get_c6(hist_norm)

    
    
    histogram.append(hist_norm)
    feature_vector=[c1,c2,c3,c4,c5,c6]
    feature_vectors.append(feature_vector)
    
feature_vectors=np.array(feature_vectors)    
print(feature_vectors.shape)

# Principle components
pca=PCA(n_components=2)
principle_components=pca.fit_transform(feature_vectors)

# svm
x_train,x_test,y_train,y_test=train_test_split(principle_components,labels,test_size=0.3,random_state=42)
classifier=SVC(decision_function_shape='ovo',kernel='linear',random_state=42)
classifier.fit(x_train,y_train)

y_pred=classifier.predict(x_test)

print("Accuracy Score",accuracy_score(y_test,y_pred))

(300, 6)
Accuracy Score 0.37777777777777777
